In [1]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
panda = Robot(verbose=False)
# data generation
X, y = load_data(robot=panda, num_samples=config.num_train_size)
# build dimension reduction model
hnne = get_hnne_model(X, y)
# get loader
loader = get_loader(X, y, hnne=hnne)
# get val loader
val_loader = get_val_loader(robot=panda, hnne=hnne)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()

hnne load successfully from /home/luca/ikpflow/weights/panda/hnne.pickle
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth


In [3]:
mu = torch.randn(size=(7,))
mu

tensor([ 2.0480, -0.1503, -1.0315, -0.1740,  1.7608, -0.2033, -0.8924])

In [4]:
mu = torch.randn(size=(7,))
print(mu)
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0)
df, err = test_l2_err(robot=panda, 
                      loader=val_loader, 
                      model=iflow, 
                      num_data=1, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

tensor([ 0.5225,  0.0390, -1.5867,  0.7135,  0.8892, -0.0216, -0.0104])


,l2_err,time_diff(per 1000)
count,1.000000,1.00
mean,0.004353,0.26
std,NaN,NaN
min,0.004353,0.26
25%,0.004353,0.26
50%,0.004353,0.26
75%,0.004353,0.26
max,0.004353,0.26


In [5]:
df, err = test_l2_err(robot=panda, 
                      loader=val_loader, 
                      model=flow, 
                      num_data=100, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

,l2_err,time_diff(per 1000)
count,100.000000,100.000000
mean,0.003999,0.101100
std,0.001941,0.004691
min,0.001950,0.100000
25%,0.002719,0.100000
50%,0.003151,0.100000
75%,0.004864,0.100000
max,0.012350,0.130000


In [6]:
nflow = get_nflow_model(flow=flow)
df, err = test_l2_err(robot=panda, 
                      loader=val_loader, 
                      model=nflow, 
                      num_data=100, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

,l2_err,time_diff(per 1000)
count,100.000000,100.000000
mean,0.003075,0.100600
std,0.004337,0.002387
min,0.001355,0.100000
25%,0.001963,0.100000
50%,0.002237,0.100000
75%,0.002948,0.100000
max,0.043407,0.110000


In [7]:
traj_dir = sample_ee_traj(robot=panda, load_time='')

100%|██████████| 20/20 [00:00<00:00, 12850.20it/s]

mkdir /home/luca/ikpflow/data/panda/trajectory/06041430
/home/luca/ikpflow/data/panda/trajectory/06041430/ load successfully.


In [8]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=nflow, robot=panda, traj_dir=traj_dir, enable_regenerate=True)

[[-0.59238789  0.33397016 -0.91804107  0.98448652]
 [-0.31615348 -0.71505912  0.09302807  0.41516754]
 [-0.1603414  -0.46532428  0.38606396  0.45180204]]
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000      20.000000
mean    0.010824 -17.724793     195.867638
std     0.019139   1.055945     111.418037
min     0.000301 -18.888065      64.115587
25%     0.002101 -18.520704      98.826977
50%     0.003717 -17.851616     188.717123
75%     0.008523 -17.388684     250.383697
max     0.080124 -14.752312     430.623155
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000      20.000000
mean    0.003006 -17.487901     115.052131
std     0.003818   0.801651      47.202314
min     0.000449 -18.517200      36.687197
25%     0.001436 -17.921692      83.809362
50%     0.002198 -17.483829     102.307293
75%     0.003102 -17.340517     138.299781
max     0.018603 -14.767996     216.607019
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000  

In [9]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=iflow, robot=panda, traj_dir=traj_dir, enable_regenerate=True)

[[-0.59245341  0.33401809 -0.91784283  0.98451249]
 [-0.33908641 -0.69984875  0.14089201  0.4359056 ]
 [-0.59238789  0.33397016 -0.91804107  0.98448652]]
          l2_err  log_prob  ang_errs(sum)
count  20.000000       0.0      20.000000
mean    0.005037       NaN      52.809407
std     0.003041       NaN      48.134939
min     0.001502       NaN       1.283280
25%     0.003321       NaN      20.126191
50%     0.004372       NaN      45.827027
75%     0.005664       NaN      69.137046
max     0.015146       NaN     194.691141
          l2_err  log_prob  ang_errs(sum)
count  20.000000       0.0      20.000000
mean    0.008960       NaN      70.788223
std     0.012011       NaN      59.930885
min     0.001437       NaN       0.415931
25%     0.002660       NaN      17.864066
50%     0.004913       NaN      60.642043
75%     0.006686       NaN     113.749574
max     0.039619       NaN     204.484225
          l2_err  log_prob  ang_errs(sum)
count  20.000000       0.0      20.000000
mean  